<a href="https://colab.research.google.com/github/iesous-kurios/powerBi/blob/master/Copia_de_PowerBI_Project_90_day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.10.0'

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/FamProm_90Day_April15.xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [0]:
#currently_enrolled.head()

In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more

from datetime import date, timedelta

current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=30)).isoformat()

need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < days_before ]

In [0]:
exit_df = need_to_exit[['3.1 FirstName', '3.1 LastName', '5.8 Personal ID', '5.9 Household ID', '3.15 Relationship to HoH', 'CaseMembers', '3.10 Enroll Date', 'Date of Last ES Stay (Beta)']].copy()

In [0]:
#exit_df.head()

In [0]:
exit_df.to_excel('/content/drive/My Drive/FPData/90 Day Data/90Dayexit_df.xlsx')



In [98]:
households = df['5.9 Household ID'].unique()
len(households)

72

In [99]:
# print total number of families served (unduplicated)
unique_households = df['5.9 Household ID'].nunique()
unique_households

72

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [101]:
avg_bn = df['Count of Bed Nights (Housing Check-ins)'].mean()
avg_bn = round(avg_bn, 2)
avg_bn

50.06

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [0]:
#df.head()

In [0]:
perm_leaver = df[df['perm_leaver'] == True]

In [0]:
total_perm = perm_leaver.perm_leaver.value_counts().sum()

In [108]:
total_perm

63

In [0]:
non_perm = df[df['perm_leaver'] == False]

In [0]:
total_non_perm = non_perm.perm_leaver.value_counts().sum()

In [111]:
total_non_perm

58

In [0]:
perm_percent = total_perm / (total_perm + total_non_perm)

In [113]:
perm_percent = round(perm_percent, 2)*100
perm_percent

52.0

In [114]:
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             44
Rental by client, other ongoing housing subsidy                                                                                  39
Rental by client, no ongoing housing subsidy                                                                                     24
Transitional Housing for homeless persons (including homeless youth)                                                              6
Staying or living with family, temporary tenure (e.g., room, apartment or house)                                                  4
Place not meant for habitation (e.g., a vehicle, an abandoned building, bus/train/subway station/airport or anywhere outside)     3
Client refused                                                                                                                    1
Name: 3.12 Exit Destination, dtype: int64

In [0]:
unknowns = ['Client refused', 'No exit interview completed', 'Other']

In [0]:
unknown_df = df[df['3.12 Exit Destination'].isin(unknowns)]

In [0]:
unknown_exits = unknown_df['3.12 Exit Destination'].value_counts().sum()

In [118]:
unknown_exits

1

In [119]:
unknown_percent = unknown_exits / total_non_perm
unknown_percent = round(unknown_percent, 2) *100
unknown_percent

2.0

In [120]:
exits = df['3.12 Exit Destination'].value_counts().sum()
exits

121

In [0]:
exits = df['3.12 Exit Destination'].value_counts()

In [122]:
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'].sum()
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
#esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))

#print('Exits to ES percent:',esExits,'%')
esExits
#print('Exits to Transitional',transExits,'%')

44

In [123]:
tempExits = df['3.12 Exit Destination'].isin(temp_reasons).sum()
temp_exit_percent = tempExits.sum() / exits.sum()
#exits.sum()) * 100)
# round tempExits to two decimals
#tempExits = round(temp_exit_percent*100, 2)
tempExits

4

In [0]:
transitional = df[df['3.12 Exit Destination'] == 'Transitional Housing for homeless persons (including homeless youth)']

In [0]:
trans_leavers = transitional['3.12 Exit Destination'].value_counts().sum()

In [126]:
trans_leavers

6

In [127]:
exits.sum()

121

In [128]:
trans_percent = (trans_leavers / exits.sum())*100
trans_percent = round(trans_percent, 2)
trans_percent

4.96

In [0]:
# export zips into an excel sheet for Power BI visual
df['V5 Zip'].value_counts().to_excel('/content/drive/My Drive/FPData/90 Day Data/90DayweeklyYTDzips.xlsx', header=True, index=True)



In [0]:
# Import pandas library 
import pandas as pd 
  
# initialize list of lists 
data = [['Permanent Leavers', total_perm], ['Emergency Shelter Leavers', esExits], 
        ['Temporary Leavers', tempExits], ['Transitional Leavers', trans_leavers],
        ['Unknown Leavers', unknown_exits]] 
  


# Create the pandas DataFrame 
power_bi = pd.DataFrame(data, columns = ['Exit Destination', 'Total']) 
  
# print dataframe. 

In [133]:
power_bi

,Exit Destination,Total
0,Permanent Leavers,63
1,Emergency Shelter Leavers,44
2,Temporary Leavers,4
3,Transitional Leavers,6
4,Unknown Leavers,1


In [0]:
power_bi.to_excel('/content/drive/My Drive/FPData/90 Day Data/90DayweeklyDataNew.xlsx')